In [ ]:
#import libraries
import librosa
import librosa.display
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from matplotlib.pyplot import specgram
import glob
from sklearn.metrics import confusion_matrix
import os
import sys
import warnings
import IPython.display as ipd
import seaborn as sns
import glob
import pickle
import json

if not sys.warnoptions :
    warnings.simplefilter('ignore')
warnings.filterwarnings("ignore", category  = DeprecationWarning)

We are taking 4 datasets, which are SAVEE dataset, RAVDESS dataset, TESS dataset and CREMA - D dataset, these all are in kaggle. 

In [ ]:
TESS = "/kaggle/input/toronto-emotional-speech-set-tess/tess toronto emotional speech set data/TESS Toronto emotional speech set data/"
RAV = "../input/ravdess-emotional-speech-audio/audio_speech_actors_01-24/"
SAVEE = "/kaggle/input/surrey-audiovisual-expressed-emotion-savee/ALL/"
CREMA = "../input/cremad/AudioWAV/"

# SAVEE DATASET

In [ ]:
dir_list = os.listdir(SAVEE)
dir_list[0:5]

#for i in dir_list:
#    print(i[-8 : -6])

SAVEE datasets
emotions are as follows
* a : anger
* d : disgust
* f : fear
* h : happiness
* sa: sadness
* su: surprise
* n : neutral

Load the dataset 

In [ ]:
#get the data location for SAVEE
dir_list = os.listdir(SAVEE)

#PARSE THE FILENAME TO GET TO EMOTIONS
emotion = []
path = []

for i in dir_list : 
    if i[-8 : -6] == "_a" : 
        emotion.append('male_angry')
    elif i[-8 : -6] == "_d" : 
        emotion.append('male_disgust')
    elif i[-8 : -6] == "_h" : 
        emotion.append('male_happy')
    elif i[-8 : -6] == "_f" : 
        emotion.append('male_fear')    
    elif i[-8 : -6] == "sa" : 
        emotion.append('male_sad')
    elif i[-8 : -6] == "su" : 
        emotion.append('male_surprise')
    elif i[-8 : -6] == "_n" : 
        emotion.append('male_neutral')
    else:
        emotion.append('male_error')
    path.append(SAVEE + i)



#check label distribution
SAVEE_df = pd.DataFrame(emotion, columns = ['labels'])
SAVEE_df['source'] = 'SAVEE'
SAVEE_df = pd.concat([SAVEE_df, pd.DataFrame(path, columns = ['path'])],axis = 1)
SAVEE_df.labels.value_counts()


# data exploration

In [ ]:
fname = SAVEE + 'KL_n27.wav'
data, sampling_rate = librosa.load(fname)
plt.figure(figsize=(15,3))
librosa.display.waveplot(data, sr = sampling_rate)
#play the audio
ipd.Audio(fname)

it can be seen from waveplot speech is clear and there is no background noice or very less noise play with other sound tracks also and check background noise and speech clearity

# RAVDEES DATASET


* Modality (01 = full-AV, 02 = video-only, 03 = audio-only).
* Vocal channel (01 = speech, 02 = song).
* Emotion (01 = neutral, 02 = calm, 03 = happy, 04 = sad, 05 = angry, 06 = fearful, 07 = disgust, 08 = surprised).
* Emotional intensity (01 = normal, 02 = strong). NOTE: There is no strong intensity for the 'neutral' emotion.
* Statement (01 = "Kids are talking by the door", 02 = "Dogs are sitting by the door").
* Repetition (01 = 1st repetition, 02 = 2nd repetition).
* Actor (01 to 24. Odd numbered actors are male, even numbered actors are female).

03-01-01-01-01-01-01.wav MEANS 03 audio only, 01 speech, 01 neutral, 01 normal intensity, 01 statement kids, 01 1st repeat and 01  means 1st actor

In [ ]:
dir_list = os.listdir(RAV)
dir_list.sort()

emotion = []
gender = []
path = []

for i in dir_list: 
    fname = os.listdir(RAV + i)
    for f in fname: 
        part = f.split(".")[0].split('-')
        emotion.append(int(part[2]))
        tmp = int(part[6])
        if tmp %2 == 0:
            tmp = 'female'
        else : 
            tmp = "male"
        gender.append(tmp)
        path.append(RAV + i + '/'+ f)
        

In [ ]:
RAV_df = pd.DataFrame(emotion)
RAV_df = RAV_df.replace({1 : 'neutral',2 : 'neutral', 3 : 'happy', 4 : 'sad',5 : 'angry',6 : 'fear',7 : 'disgust',8 : 'surprise' })
RAV_df = pd.concat([pd.DataFrame(gender), RAV_df], axis = 1)
RAV_df.columns = ['gender','emotion']
RAV_df['labels'] = RAV_df.gender + "_" + RAV_df.emotion
RAV_df = pd.concat([RAV_df, pd.DataFrame(path, columns = ['path'])], axis = 1)
RAV_df.drop(['gender','emotion'], axis = 1)
RAV_df.labels.value_counts()

# Data Exploration

In [ ]:
fname = RAV + 'Actor_04/03-01-01-01-01-02-04.wav'
data, sampling_rate = librosa.load(fname)
plt.figure(figsize=(15,3))
librosa.display.waveplot(data, sr = sampling_rate)
#play the audio
ipd.Audio(fname)

As we can see from the waveform that, it is preety clear and less of background of noise and on the beginning and the end there is no speech so it is clear. 

# TESS DATASET


Now load the dataset

In [ ]:
dir_list = os.listdir(TESS)
dir_list.sort()
dir_list

In [ ]:
path = []
emotion = []
gender = []


for i in dir_list:
    fname = os.listdir(TESS + i)
    for f in fname: 
        part = f.split(".")[0].split('_')
        tmp = part[0]
        if tmp == 'OAF':
            tmp = 'male'
        else : 
            tmp = 'female'
        gender.append(tmp)
        emotion.append(part[2])
        path.append(TESS + i + '/' + f)

TESS_df = pd.DataFrame(emotion)
TESS_df = pd.concat([pd.DataFrame(gender), TESS_df], axis = 1)
TESS_df.columns = ['gender','emotion']
TESS_df['labels'] = TESS_df.gender + "_" + TESS_df.emotion
TESS_df['source'] = TESS
TESS_df = pd.concat([TESS_df, pd.DataFrame(path,columns = ['path'])], axis = 1)
TESS_df = TESS_df.drop(['gender', 'emotion','source'], axis = 1)
TESS_df["labels"].replace({"male_ps": "male_surprise", "female_ps": "female_surprise"}, inplace=True)
TESS_df.labels.value_counts()
        
        
        

# Data Exploration

In [ ]:
fname = TESS + '/OAF_Sad/OAF_bar_sad.wav'
data, sampling_rate = librosa.load(fname)
plt.figure(figsize=(15,3))
librosa.display.waveplot(data, sr = sampling_rate)
#play the audio
ipd.Audio(fname)

# CREMA D Dataset

In [ ]:
dir_list = os.listdir(CREMA)
dir_list.sort()
print(dir_list[0:10])

In [ ]:
gender = []
emotion = []
path = []
female = [1002,1003,1004,1006,1007,1008,1009,1010,1012,1013,1018,1020,1021,1024,1025,1028,1029,1030,1037,1043,1046,1047,1049,
          1052,1053,1054,1055,1056,1058,1060,1061,1063,1072,1073,1074,1075,1076,1078,1079,1082,1084,1089,1091]

for i in dir_list: 
    part = i.split('_')
    if int(part[0]) in female:
        temp = 'female'
    else:
        temp = 'male'
    gender.append(temp)
    if part[2] == 'SAD' and temp == 'male':
        emotion.append('male_sad')
    elif part[2] == 'ANG' and temp == 'male':
        emotion.append('male_angry')
    elif part[2] == 'DIS' and temp == 'male':
        emotion.append('male_disgust')
    elif part[2] == 'FEA' and temp == 'male':
        emotion.append('male_fear')
    elif part[2] == 'HAP' and temp == 'male':
        emotion.append('male_happy')
    elif part[2] == 'NEU' and temp == 'male':
        emotion.append('male_neutral')
    elif part[2] == 'SAD' and temp == 'female':
        emotion.append('female_sad')
    elif part[2] == 'ANG' and temp == 'female':
        emotion.append('female_angry')
    elif part[2] == 'DIS' and temp == 'female':
        emotion.append('female_disgust')
    elif part[2] == 'FEA' and temp == 'female':
        emotion.append('female_fear')
    elif part[2] == 'HAP' and temp == 'female':
        emotion.append('female_happy')
    elif part[2] == 'NEU' and temp == 'female':
        emotion.append('female_neutral')
    else:
        emotion.append('Unknown')
    path.append(CREMA + i)
    
CREMA_df = pd.DataFrame(emotion, columns = ['labels'])
CREMA_df['source'] = 'CREMA'
CREMA_df = pd.concat([CREMA_df,pd.DataFrame(path, columns = ['path'])],axis=1)
CREMA_df.labels.value_counts()

# Data Exploration

In [ ]:
fname = CREMA + '/1001_DFA_SAD_XX.wav'
data, sampling_rate = librosa.load(fname)
plt.figure(figsize=(15,3))
librosa.display.waveplot(data, sr = sampling_rate)
#play the audio
ipd.Audio(fname)

In [ ]:
df = pd.concat([SAVEE_df, RAV_df, TESS_df, CREMA_df], axis = 0)
df.drop(['gender','emotion'], axis = 1)
print(df.labels.value_counts())
df.to_csv("Data_path1.csv",index=False)

# Feature Extraction

There are two type of features 1. Time domain and 2nd is Frequency domain feature 

Time domain are simple to extract and understand, like energy of signal, zero crossing rate, maximum amplitude and minimum energy. Whereas frequency domain features are obtained by converting the time based signal into frequency domain and they are harder to comprehend and it provides the extra information like pitch , rhymes, meloduy etc. 

# MFCC

Mel frequency ceptral cofficient it is a good representation of vocal tract like X ray.  In machine learning applications MFCC are treated as the image representation of sound track and it can give more information and it has an ability to draw on transfer learning. 

In [ ]:
#checking out MFCC 
#source : RAVDESS, Gender  : Female and Emotion : Angry
path = "../input/ravdess-emotional-speech-audio/Actor_08/03-01-05-01-01-01-08.wav"
X, sample_rate = librosa.load(path, res_type= 'kaiser_fast', duration = 2.5,sr = 22050*2, offset = 0.5)
mfcc = librosa.feature.mfcc(y = X, sr = sample_rate, n_mfcc = 13)

#audio wave graph

plt.figure(figsize = (10,5))
plt.subplot(3,1,1)
librosa.display.waveplot(X, sr = sample_rate)
plt.title('audio sampled at 44100 hrz')

#MFCC

plt.figure(figsize = (10,5))
plt.subplot(3,1,1)
librosa.display.specshow(mfcc, x_axis = 'time')
plt.ylabel('MFCC')
plt.colorbar()

ipd.Audio(path)

In [ ]:
path = "../input/ravdess-emotional-speech-audio/Actor_06/03-01-03-01-01-01-06.wav"
X, sample_rate = librosa.load(path, res_type = 'kaiser_fast', duration = 2.5, sr = 22050*2, offset = 0.5)
mfcc = librosa.feature.mfcc(y = X, sr = sample_rate, n_mfcc = 13)

#audio waveform
plt.figure(figsize=(10,5))
plt.subplot(3,1,1)
librosa.display.waveplot(X, sr = sample_rate)
plt.title("audio sampled at 44100 hz")

#mfcc waveforms
plt.figure(figsize = (10,5))
plt.subplot(3,1,1)
librosa.display.specshow(mfcc, x_axis = 'time')
plt.ylabel('MFCC')

ipd.Audio(path)

# statistical features

here we are taking a mean across the each band over time and it will give us distinctive feature. if we see the MFCC plot, first band at the bottom is the most distinctive band and we are taking the shorter time window it means that, changes observed overtime does not vary greatly. So key feature is to capturing the information contained in the various bands and display it as a time series plot to illustrate the point.

In [ ]:
# comparing male and female angry for the same sentence uttered. 
path = "../input/ravdess-emotional-speech-audio/Actor_08/03-01-05-01-01-01-08.wav"
X, sample_rate = librosa.load(path, res_type= 'kaiser_fast', duration = 2.5, sr = 22050*2, offset = 0.5)
female = librosa.feature.mfcc(y= X, sr = sample_rate, n_mfcc = 13)
female = np.mean(librosa.feature.mfcc(y = X, sr = sample_rate, n_mfcc = 13), axis = 0)
print(len(female))

# comparing male and female angry for the same sentence uttered. 
path = "../input/ravdess-emotional-speech-audio/Actor_06/03-01-03-01-01-01-06.wav"
X, sample_rate = librosa.load(path, res_type= 'kaiser_fast', duration = 2.5, sr = 22050*2, offset = 0.5)
male = librosa.feature.mfcc(y= X, sr = sample_rate, n_mfcc = 13)
male = np.mean(librosa.feature.mfcc(y = X, sr = sample_rate, n_mfcc = 13), axis = 0)
print(len(male))

#audio wave
plt.figure(figsize = (20,15))
plt.subplot(3,1,1)
plt.plot(female, label = 'female')

plt.plot(male, label = 'male')
plt.legend()

here we can see that for the same sentence spoken by male and female, female has higher pitch. 

## Model Creation 

Importing required libraries

In [ ]:
import keras
from keras import regularizers
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, Model, model_from_json
from keras.layers import Dense, Embedding, LSTM
from keras.layers import Input, Flatten, Dropout, Activation, BatchNormalization
from keras.layers import Conv1D, MaxPool1D, AveragePooling1D
from keras.utils import np_utils, to_categorical
from keras.callbacks import ModelCheckpoint

#sklearn
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder


In [ ]:
ref = pd.read_csv('./Data_path1.csv')
ref.drop(['gender','emotion'], axis = 1, inplace = True)
ref.head()
print(ref.labels.value_counts())
ref.to_csv("Data_path2.csv",index=False)

now we will read all the audio files , extract its mean across all MFCC bands by time and just keep extracting the features, dropping entire audio file data

In [ ]:
ref1 = pd.read_csv('./Data_path2.csv')
ref1.head()

In [ ]:
df = pd.DataFrame(columns = ['features'])
# looping feature extraction over entire dataset

counter = 0
for index,path in enumerate(ref1.path):
    X, sample_rate = librosa.load(path, res_type='kaiser_fast', duration = 2.5, sr = 22050*2, offset = 0.5)
    sample_rate = np.array(sample_rate)
    
    #taking mean of MFCC 
    mfccs = np.mean(librosa.feature.mfcc(y = X, sr = sampling_rate, n_mfcc = 13), axis = 0)
    df.loc[counter] = [mfccs]
    counter = counter+1 
print(len(df))
df.head()

In [ ]:
df = pd.concat([ref1, pd.DataFrame(df['features']. values.tolist())], axis = 1)
df[:5]

In [ ]:
df = df.fillna(0)
print(df.shape)
df[:5]

Splitting the data in 2 parts: training and validation and measuring models performance and accuracy 

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(['path','labels','source'], axis = 1), df.labels, test_size = 0.25, shuffle = True, random_state = 42)

In [ ]:
X_train[150 : 160]

Now we will convert the data format to numpy array and implement a 1D CNN

In [ ]:
X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)

#one hot encode target 
lb = LabelEncoder()
y_train = np_utils.to_categorical(lb.fit_transform(y_train))
y_test = np_utils.to_categorical(lb.fit_transform(y_test))

print(X_train.shape)
print(lb.classes_)

#pickle lb object for future use

filename = 'labels'
outfile = open(filename, 'wb')
pickle.dump(lb,outfile)
outfile.close()

we are using 1D CNN, so we need to expand our dimention to 3rd dimention

In [ ]:
X_train = np.expand_dims(X_train, axis = 2)
X_train.shape

In [ ]:
X_test = np.expand_dims(X_test, axis = 2)
X_test.shape

# Creating Model

In [ ]:
#New Model

model = Sequential()
model.add(Conv1D(256, 8, padding = 'same', input_shape = (X_train.shape[1],1))) 
model.add(Activation('relu'))
model.add(Conv1D(256,8,padding = 'same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(MaxPool1D(pool_size = (8)))
model.add(Conv1D(128, 8 , padding = 'same'))
model.add(Activation('relu'))
model.add(Conv1D(128, 8 , padding = 'same'))
model.add(Activation('relu'))
model.add(Conv1D(128, 8 , padding = 'same'))
model.add(Activation('relu'))
model.add(Conv1D(128, 8 , padding = 'same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(MaxPool1D(pool_size = (8)))
model.add(Conv1D(64, 8 , padding = 'same'))
model.add(Activation('relu'))
model.add(Conv1D(64, 8 , padding = 'same'))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(14)) # no of target class
model.add(Activation('softmax'))

model.summary()

In [ ]:
opt = keras.optimizers.RMSprop(learning_rate = 0.00001, decay = 1e-6)
model.compile(loss = 'categorical_crossentropy', optimizer = opt, metrics = ['accuracy'])
history = model.fit(X_train,y_train, batch_size = 16, epochs = 100, validation_data = (X_test, y_test))

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title("model loss")
plt.ylabel("loss")
plt.xlabel('epochs')
plt.legend(['train','test'],loc = 'upper left')
plt.show()

# save the model

In [ ]:
model_name = 'Emotion_recognition.h5'
save_dir = os.path.join(os.getcwd(),'saved_models')
if not os.path.isdir(save_dir):
    os.mkdir(save_dir)
model_path = os.path.join(save_dir,model_name)
model.save(model_path)
print("save model and weight at %s" %model_path)
#save model to disk
model_json = model.to_json()
with open("model_json.json", "w") as file:
    file.write(model_json)